In [1]:
import igl # Library to load meshes and perform operations on them
import meshplot as mp # Library to visualize meshes and point clouds
import vedo as vd # Library to visualize meshes and point clouds
import polyscope as ps # Library to visualize meshes
import numpy as np # Library to perform operations on matrices
import os # Library to perform operations on files and directories
import matplotlib.pyplot as plt
from math import pi

# Importing the classes and functions from the visualization folder
vd.settings.default_backend = 'k3d'

# Directory path
dir_path = os.getcwd()

In [16]:
v, f = igl.read_triangle_mesh("C:/Users/aikyna/Desktop/hananLab/hJupyter/models/Extra/botanic-garden-parallelogram-0.obj")
V = len(v) # Number of vertices
F = len(f) # Number of faces
print(F)

v1, v2, k1, k2 = igl.principal_curvature(v, f)
normals = igl.per_vertex_normals(v, f)
h2 = 0.5 * (k1 + k2)


2152


In [17]:
center_abs_init = np.zeros(3)

matrix_s = np.zeros((3 * V, 3))
vector_b = np.zeros(3*V)
for v_ind in range(V):
    normal = normals[v_ind]
    matrix_s[3 * v_ind, :] = np.array([0, -normal[2], normal[1]])
    matrix_s[3 * v_ind + 1, :] = np.array([normal[2], 0, -normal[0]])
    matrix_s[3 * v_ind + 2, :] = np.array([-normal[1], normal[0], 0])
    vector_b[3 * v_ind:3 * v_ind + 3] = matrix_s[3 * v_ind:3 * v_ind + 3, :] @ v[v_ind]
#print(matrix_s.T@matrix_s)
#print(matrix_s.T@vector_b)

center_abs_init = np.linalg.lstsq(matrix_s.T@matrix_s, matrix_s.T@vector_b, rcond=None)[0]
print(center_abs_init)

[-1.72103916  3.26808518 -4.57609306]


In [18]:
p = mp.plot(v, f, return_plot=True);
p.add_lines(v, v + normals, shading={"line_color": "red"});
p.add_points(np.array([center_abs_init]), shading={"point_color": "green", "point_size": 10.0});


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.436749…

In [19]:
def compute_J(v, normals, w, indic, X, J, r, p_length):
    center_var = X[0:3]
    for v_ind in range(V):
        vec = v[v_ind] - center_var
        mu = X[3 + v_ind]
        J[v_ind, 0:3] = -np.dot(vec, normals[v_ind]) * (-2 * np.dot(vec, vec) * normals[v_ind] + 2 * vec * np.dot(vec, normals[v_ind])) / (np.dot(vec, vec) ** 2)
        r[v_ind] = 1 - np.dot(vec, normals[v_ind]) ** 2 / np.dot(vec, vec)
        J[v_ind, :] *= w[v_ind]
        r[v_ind] *= w[v_ind]
        if indic == 1:
            angle = np.arccos(np.dot(vec, normals[v_ind]) / np.linalg.norm(vec))
            w[v_ind] = 1 / (np.tan(angle) ** 2 + 1)

        #J[v_ind + V, 0:3] = -normals[v_ind] *(1 / p_length[v_ind])
        #J[v_ind + V, 3 + v_ind] = -2 * mu
        #r[v_ind + V] = np.dot(vec, normals[v_ind]) / p_length[v_ind] - mu ** 2
        #p_length[v_ind] = np.linalg.norm(vec)

In [20]:
def optimization_abs_center(v, V, normals, center_abs_init, indic):
    J = np.zeros((2 * V, 3 + V))
    r = np.zeros(2 * V)
    X_0 = np.concatenate((center_abs_init, np.random.rand(V)))
    w = np.ones(V)
    p_length = np.zeros(V)
    for v_ind in range(V):
        p_length[v_ind] = np.linalg.norm(v[v_ind] - center_abs_init)
        
    from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, linalg
    # Compute pseudo Hessian
    X = X_0

    for i in range(20):
        compute_J(v, normals, w, indic, X, J, r, p_length)    

        H = J.T@J

        H[np.diag_indices_from(H)] += np.diag(H).max()*1e-6

        # Sparse matrix H
        H = csc_matrix(H)

        # Solve for dx
        dx = linalg.spsolve(H, -J.T@r)
        # Update vertices
        X = X + 0.9*dx
        
        # energy
        energy = r.T@r
        print(f"energy: {energy}\t dx: {np.linalg.norm(dx)}")
    return X[0:3]

In [21]:
opt_point1 = optimization_abs_center(v, V, normals, center_abs_init, 0)
opt_point2 = optimization_abs_center(v, V, normals, center_abs_init, 1)
print(opt_point1, opt_point2)

energy: 41.955186898202484	 dx: 7.590151365729734
energy: 31.06725700172344	 dx: 4.778932665190256
energy: 32.94440324061256	 dx: 9.53841622403151
energy: 35.935187884089686	 dx: 10.881466000235367
energy: 36.55176386241608	 dx: 9.214929529559413
energy: 33.82883525784347	 dx: 8.488427131628757
energy: 33.998547601047925	 dx: 9.594308661112875
energy: 35.6271009128474	 dx: 9.87140262448777
energy: 35.072146757983546	 dx: 9.580767405868727
energy: 35.24295641977788	 dx: 9.346809432928108
energy: 34.31217170258823	 dx: 9.653557044995527
energy: 35.767104262768854	 dx: 9.743382688517325
energy: 34.65112034535263	 dx: 9.648888969750601
energy: 35.628670080820214	 dx: 9.58378008393892
energy: 34.4396888946376	 dx: 9.663555582651131
energy: 35.7652531292122	 dx: 9.690358314029085
energy: 34.53843172524974	 dx: 9.661462810457975
energy: 35.72128967010754	 dx: 9.643620680037536
energy: 34.47999086657945	 dx: 9.664944952094373
energy: 35.75757984206231	 dx: 9.672676654006851
energy: 41.95518689

In [22]:
p = mp.plot(v, f, return_plot=True);
p.add_lines(v, v + normals, shading={"line_color": "red"});
p.add_points(np.array([center_abs_init]), shading={"point_color": "red", "point_size": 10.0});
p.add_points(np.array([opt_point1]), shading={"point_color": "green", "point_size": 10.0});
p.add_points(np.array([opt_point2]), shading={"point_color": "purple", "point_size": 10.0});

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.436749…

In [23]:
angles_init = np.zeros(V)
angles_1 = np.zeros(V)
angles_2 = np.zeros(V)
for v_ind in range(V):
    angles_init[v_ind] = np.arccos(np.dot(v[v_ind] - center_abs_init, normals[v_ind]) / np.linalg.norm(v[v_ind] - center_abs_init)) * 180 / np.pi
    angles_1[v_ind] = np.arccos(np.dot(v[v_ind] - opt_point1, normals[v_ind]) / np.linalg.norm(v[v_ind] - opt_point1)) * 180 / np.pi
    angles_2[v_ind] = np.arccos(np.dot(v[v_ind] - opt_point2, normals[v_ind]) / np.linalg.norm(v[v_ind] - opt_point2)) * 180 / np.pi

# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.hist(angles_init, alpha = 0.7, color = 'g')
# ax.hist(angles_1, alpha = 0.6, color = 'b')
# ax.hist(angles_2, alpha = 0.5, color = 'r')

# p = mp.plot(v, f, c = angles2,  return_plot=True);
# p.add_lines(v, v + vector, shading={"line_color": "red"});
# p.add_points(np.array([center_abs_init]), shading={"point_color": "red", "point_size": 10.0});
# p.add_points(np.array([opt_point]), shading={"point_color": "green", "point_size": 10.0});

In [24]:
# ps.init()
# ps.remove_all_structures()

# mesh = ps.register_surface_mesh("Surf", v, f)

# mesh.add_scalar_quantity("initial", angles_init)
# mesh.add_scalar_quantity("w/o weights", angles_1)
# mesh.add_scalar_quantity("with weights", angles_2)

# ps.show()

In [25]:
center_abs = opt_point1

file_export = open("C:/Users/aikyna/Desktop/Projects/Meshes_with_Spherical_faces/Omega.dat", "w")

# Export face neighbors to .dat file
file_export.write(f"{center_abs[0]} {center_abs[1]} {center_abs[2]}\t")
file_export.close()
